In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [908 kB]
Hit:13 http://ppa.

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-11 06:17:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-03-11 06:17:32 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()
# spark = SparkSession.builder.appName("BigData-HW-1").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://big-data-etl.s3.amazonaws.com/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv"), sep='\t')

# Show DataFrame
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [6]:
# Get the number of rows in the DataFrame.
df.count()

2557288

# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

# review_id_df = df.select('review_id','customer_id','product_id','product_parent',to_date('review_date','yyyy-mm-dd'))
review_id_df = df.select('review_id','customer_id','product_id','product_parent',to_date(df.review_date).alias('review_date'))
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RED72VWWCOS7S|   32787517|B008HDQYLQ|     348668413| 2015-08-31|
| RZHWQ208LTEPV|   16374060|B005OBZBD6|     264704759| 2015-08-31|
|R37LBC3XAVLYOO|    9984817|B00RQL8U2G|      95173602| 2015-08-31|
|R3L7XJMA0MVJWC|   12635190|B0081SBO4Y|     835659279| 2015-08-31|
|R2I2GHSI7T1UBN|   43905102|B008E6OK3U|     539243347| 2015-08-31|
|R2GFFKHK4I6VMX|   52596997|B00W6NTULY|     337446474| 2015-08-31|
|R1R0UDX2XAN1S4|   43871104|B00GXUMYKA|     468857193| 2015-08-31|
|R22C8FMBSTFRY8|   11346008|B005EIX8JS|     125753094| 2015-08-31|
|R118NNIQ75XPGO|   49206471|B000HJBKMQ|     834273114| 2015-08-31|
|R30HYXHZQ49621|   37596267|B004LY59V6|     612086079| 2015-08-31|
|R3EMLKY0GF1E90|   31554283|B00CAVM85M|     280334010| 2015-08-31|
|R23BX7EGJMGQJR|   43211735|B00DP6X1LG|     233116679| 2015-08

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
print("Original Dataframe count: ", df.count())
products_df = df.select('product_id','product_title').dropDuplicates(['product_id'])
print("New Dataframe count: ", products_df.count())

products_df.show()


Original Dataframe count:  2557288
New Dataframe count:  282068
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0961885629|Renee's Garden Re...|
|0982305427|Curry Tree Seeds ...|
|0989523802|Sleepytime Press ...|
|144072007X|Sand Shark Margar...|
|1554701503|Napoleon's Everyd...|
|1602003920|Resurrection Eggs...|
|1609960459|Stop Sign Two-Sid...|
|1754164498|SpaGuard Enhanced...|
|1846814618|A Practical Guide...|
|1880241064|Bird Watchers Dig...|
|1887148914|Thayer Birding So...|
|3164100000|Gadsden Flag - 2'...|
|3866814550|Fritz Powerbook 2...|
|386681481X|Typical Mistakes ...|
|398501938X|Handheld Electron...|
|5555000014|Accutire Digital ...|
|6035000029|All Weather Heavy...|
|6035000045|13 Foot Market Pa...|
|604917377X|Gustav The Gargoy...|
|7301045328|CBconcept 120VLR1...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
from pyspark.sql.functions import count
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
# customers_df = df.groupby('customer_id').agg({"customer_id":"count"}.alias("customer_count"))
customers_df = df.groupby('customer_id').agg(count("customer_id").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45046643|             2|
|   37757447|             3|
|   27567982|             2|
|   29135708|             3|
|   50583551|             7|
|    2696024|             1|
|    2447629|             1|
|     641157|             1|
|   10245571|             4|
|   42900950|             1|
|   51506042|             1|
|   35665618|             6|
|   16092109|             1|
|    3223564|             1|
|   45506282|             1|
|    4061339|             1|
|   27484443|             1|
|   30238476|             1|
|   36516743|             7|
|   17059277|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
from pyspark.sql.types import IntegerType
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
# vine_df = df.select('review_id', \
#                     col('star_rating').cast("int").as("star_rating"), \
#                     col('helpful_votes').cast("int").as("helpful_votes"), \
#                     col('total_vines').cast("int").as("total_vines"))

vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df = vine_df.withColumn('star_rating', vine_df['star_rating'].cast(IntegerType()))
vine_df = vine_df.withColumn('helpful_votes', vine_df['helpful_votes'].cast(IntegerType()))
vine_df = vine_df.withColumn('total_votes', vine_df['total_votes'].cast(IntegerType()))

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RED72VWWCOS7S|          1|            2|          8|   N|
| RZHWQ208LTEPV|          5|            0|          0|   N|
|R37LBC3XAVLYOO|          5|            4|          5|   N|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|
|R22C8FMBSTFRY8|          5|            2|          2|   N|
|R118NNIQ75XPGO|          3|            0|          0|   N|
|R30HYXHZQ49621|          2|            0|          0|   N|
|R3EMLKY0GF1E90|          5|            0|          0|   N|
|R23BX7EGJMGQJR|          5|            1|          2|   N|
|R2Z4B6SDEAZF6E|          5|            0|          0|   N|
|R35289PGJERP5J|          5|            

# Load

In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<RDS database>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [12]:
# Print all schemas
print("Review_ID Schema:")
print(review_id_df.printSchema())

print("Products")
print(products_df.printSchema())

print("Customers")
print(customers_df.printSchema())

print("Vine")
print(vine_df.printSchema())

Review_ID Schema:
root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)

None
Products
root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

None
Customers
root
 |-- customer_id: string (nullable = true)
 |-- customer_count: long (nullable = false)

None
Vine
root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

None


In [13]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='lawn_review', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='lawn_products', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='lawn_customers', mode=mode, properties=config)

In [16]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='lawn_vine', mode=mode, properties=config)